# PlayStore Crawler
## 날짜 : 2021-03-17 수요일
## 저자 : 이준호, 최인서, 이승수, 영모
### 참조 : https://signing.tistory.com/44?category=417668

# 목차
## 1. URL open, data crawling test
## 2. 대규모 crawling, pandas에 넣기
## 3. Data check 및 전처리
## 4. Data csv file로 save

## 1. URL open, data crawling test
## !! URL 및 chromedriver path 수정하기 !!

In [65]:
# selenium import
from selenium import webdriver
# !!수정하기!! 접속 시도하려는 URL 입력
# url = 'https://play.google.com/store/apps/details?id=com.stx.pullhimout&showAllReviews=true'
url = 'https://play.google.com/store/apps/details?id=com.epicactiononline.ffxv.ane&hl=ko&gl=US&showAllReviews=true'
# !!수정하기!! 크롬 드라이버 경로
# windows는 ./chromedriver.exe 로 수정
driverPath = "./chromedriver" # Chrome Driver path
driver = webdriver.Chrome(driverPath)   # Open Chrome 
driver.set_window_size(1400, 800)
driver.get(url)

In [38]:
# url = 'https://play.google.com/store/apps/details?id=com.devsisters.ck'
# driver.get(url)

In [21]:
# # 검색창에 게임 검색하기
# name = '메이플스토리'
# driver.find_element_by_xpath("//*[@id='gbqfq']").send_keys(name)
# driver.find_element_by_xpath("//*[@id='gbqfb']").click()

# # 검색창에 나온 앱중 가장 첫번째거 클릭하기
# driver.find_element_by_xpath('//*[@id="fcxH9b"]').click()

In [ ]:
# 전체 리뷰 보기

In [66]:
# 리뷰 data를 xpath로 찾아서 긁어오기
review = driver.find_element_by_xpath("//span[@jsname='bN97Pc']")
date = driver.find_element_by_xpath("//span[@class='p2TkOb']")
star = driver.find_element_by_xpath("//span[@class='nt2C1d']/div[@class='pf5lIe']/div[@role='img']") 

# 읽어온 data 출력
print("리뷰 데이터:", review.text)
print("날짜 데이터:", date.text)
print("별점 데이터:", star.get_attribute('aria-label'))

리뷰 데이터: 시간을 아~주 오래들이면 노현질로도 충분히 즐길수있는게임 처음엔 약탈당하고 어려울수있으나 길드원들에게 도움청하면 아주잘 도와주심 무과금으로 1년 넘게해봤는데 충분히 싸우고 즐길수있음 요샌 이벤트를 퍼주고있어서 부계정 새로키웠는데 본계정만큼 크고있음ㅇㅅㅇ
날짜 데이터: 2021년 2월 23일
별점 데이터: 별표 5개 만점에 5개를 받았습니다.


## 2. 대규모 crawling, pandas에 넣기

In [67]:
# time 라이브러리 import
import time
SCROLL_PAUSE_TIME = 1.5
last_height = driver.execute_script("return document.body.scrollHeight")

# review 긁어오는 횟수, count x 40개 긁어옴
count = 12

while count > 0:
    # (1) 4번의 스크롤링
    for i in range(5):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
    # (2) 더보기 클릭
    driver.find_element_by_xpath("//span[@class='RveJvd snByac']").click()
    # 카운트 세는 용도
    count -= 1
    # (3) 종료 조건
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [68]:
# crawling data 불러오기
reviews = driver.find_elements_by_xpath("//span[@jsname='bN97Pc']")
dates = driver.find_elements_by_xpath("//span[@class='p2TkOb']")
stars = driver.find_elements_by_xpath("//span[@class='nt2C1d']/div[@class='pf5lIe']/div[@role='img']") 

In [69]:
# crawling data 길이 확인
len(reviews)

2000

In [70]:
# crawling한 data를 pandas에 넣기
import pandas as pd

res_dict = [] 
for i in range(len(reviews)): 
    res_dict.append({ 
        'DATE' : dates[i].text,
        'STAR' : stars[i].get_attribute('aria-label'), 
        'REVIEW' : reviews[i].text }) 
    res_df = pd.DataFrame(res_dict) 

# dataframe 확인
res_df

,DATE,STAR,REVIEW
0,2021년 2월 23일,별표 5개 만점에 5개를 받았습니다.,시간을 아~주 오래들이면 노현질로도 충분히 즐길수있는게임 처음엔 약탈당하고 어려울수...
1,2021년 1월 22일,별표 5개 만점에 1개를 받았습니다.,"혹여나 이벤 때문에 시작하려한다면, 하지마세요. 현질 유도게임입니다. 게임에 현질안..."
2,2021년 1월 16일,별표 5개 만점에 1개를 받았습니다.,"돈없음 절대,절~~~대 할 수없는 게임...시작이 후회되는 게임입니다...길드가입해..."
3,2021년 2월 5일,별표 5개 만점에 2개를 받았습니다.,ㅋㅋ 게임 자체는 그냥 평범 그 자체인데 이벤트 돈 안줘서 평점 겁나 깎이네ㅋㅋㅋ ...
4,2021년 1월 16일,별표 5개 만점에 1개를 받았습니다.,솔로이벤트 보상으로 모그리 크레딧이 계속 안들어오네요. 여태 못 받은것만 몇천 크레...
...,...,...,...
1995,2019년 3월 4일,별표 5개 만점에 3개를 받았습니다.,10레벨 달성했어요 카카오페이 왜 안줘여?
1996,2018년 11월 7일,별표 5개 만점에 1개를 받았습니다.,ㄹㅇ 카카오페이지아니였음 안했을 최악에게임
1997,2018년 12월 30일,별표 5개 만점에 1개를 받았습니다.,성체10달성시 케쉬 증정한다던데 안주나요? 사기꾼들 아직 홍보하네..
1998,2019년 1월 12일,별표 5개 만점에 1개를 받았습니다.,10레벨 올리면 한게임 21억 준다며 구라치지말고 21억 내놔라


In [71]:
# dataframe 확인
res_df.head(5)

,DATE,STAR,REVIEW
0,2021년 2월 23일,별표 5개 만점에 5개를 받았습니다.,시간을 아~주 오래들이면 노현질로도 충분히 즐길수있는게임 처음엔 약탈당하고 어려울수...
1,2021년 1월 22일,별표 5개 만점에 1개를 받았습니다.,"혹여나 이벤 때문에 시작하려한다면, 하지마세요. 현질 유도게임입니다. 게임에 현질안..."
2,2021년 1월 16일,별표 5개 만점에 1개를 받았습니다.,"돈없음 절대,절~~~대 할 수없는 게임...시작이 후회되는 게임입니다...길드가입해..."
3,2021년 2월 5일,별표 5개 만점에 2개를 받았습니다.,ㅋㅋ 게임 자체는 그냥 평범 그 자체인데 이벤트 돈 안줘서 평점 겁나 깎이네ㅋㅋㅋ ...
4,2021년 1월 16일,별표 5개 만점에 1개를 받았습니다.,솔로이벤트 보상으로 모그리 크레딧이 계속 안들어오네요. 여태 못 받은것만 몇천 크레...


## 3. Data check 및 전처리

In [72]:
# 별점 data 숫자로 전처리하기
df = []
for i in range(len(res_df['STAR'])):
    a = res_df['STAR'][i][10:11]
    df.append(a)

res_df['star'] = df
res_df.drop('STAR', axis=1, inplace=True)

In [73]:
# 날짜 data 전처리
for i in range(len(res_df['DATE'])):
    res_df['DATE'][i] = res_df['DATE'][i].replace('년 ', '-').replace('월 ', '-').replace('일', '')

In [74]:
# 전처리 data 확인
res_df.tail(5)

,DATE,REVIEW,star
1995,2019-3-4,10레벨 달성했어요 카카오페이 왜 안줘여?,3
1996,2018-11-7,ㄹㅇ 카카오페이지아니였음 안했을 최악에게임,1
1997,2018-12-30,성체10달성시 케쉬 증정한다던데 안주나요? 사기꾼들 아직 홍보하네..,1
1998,2019-1-12,10레벨 올리면 한게임 21억 준다며 구라치지말고 21억 내놔라,1
1999,2020-12-25,렉이 심함,3


## 4. Data csv file로 save
## !! 따옴표 안에 게임 이름.csv로 변경 !!

In [75]:
# data csv file로 저장
# !! 따옴표 안에 게임 이름으로 변경 !!
res_df.to_csv('ffxv.csv')

In [76]:
# chrome driver 종료
driver.close()